In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

In [ ]:
data = pd.read_csv("BTC-USD_SIGNALS.csv")
data

# Correlations

In [ ]:
corr = data.corr("spearman")
corr

In [ ]:
corr[["Variation"]].sort_values(by="Variation")

In [ ]:
pd.plotting.scatter_matrix(data,figsize=[12,12])

## Adding Variation+1

In [ ]:
newdata = data.copy()
newdata["Variation+1"] = data["Variation"].shift(-1)
newdata

In [ ]:
corr = newdata.corr("spearman")
corr

In [ ]:
corr[["Variation+1"]].sort_values(by="Variation+1")

In [ ]:
#plt.show()

## Variation with respect to other variables

In [ ]:
plt.figure(figsize=[10,5])
plt.scatter(data["RSI"],data["Variation"])

In [ ]:
plt.figure(figsize=[10,5])
plt.scatter(data["MACD_H"],data["Variation"])

In [ ]:
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')
ax.scatter(list(data["MACD_H"]),list(data["RSI"]),list(data["Variation"]))
plt.xlabel(xlabel="RSI")
plt.ylabel(ylabel="MACD_H")
plt.legend()
plt.show()

# Examining Predict Data

## Initial Data

In [ ]:
predict_data = data.copy().drop(["Open","Close","High","Low"],axis=1)
max_days = 5
target_range = 2
for i in range(1,max_days):#2jours
    #predict_data[["Variation-{}".format(i),"Vol-{}".format(i),"RSI-{}".format(i),"MACD-{}".format(i),"MACD_H-{}".format(i),"CONF-{}".format(i),"TRANS-{}".format(i),"REV-{}".format(i),"FnG-{}".format(i)]] = data[["Variation","Volume","RSI","MACD","MACD_H","Confirmation Time","Transactions","Miners Revenue","FnG"]].shift(i)
    predict_data[["Variation-{}".format(i),"Vol-{}".format(i),"RSI-{}".format(i),"MACD-{}".format(i),"MACD_H-{}".format(i),"CONF-{}".format(i),"TRANS-{}".format(i),"REV-{}".format(i),"FnG-{}".format(i), "ADX-{}".format(i), "+DM-{}".format(i), "-DM-{}".format(i)]] = data[["Variation","Volume","RSI","MACD","MACD_H","Confirmation Time","Transactions","Miners Revenue","FnG","ADX14","+DM","-DM"]].shift(i)
    #predict_data[["Variation-{}".format(i),"Vol-{}".format(i),"RSI-{}".format(i),"MACD-{}".format(i),"MACD_H-{}".format(i)]] = data[["Variation","Volume","RSI","MACD","MACD_H"]].shift(i)
#predict_data["Target"] = (data["Variation"].shift(-1) >= 0)
predict_data["Target"] = (data["Close"].shift(-target_range) - data["Close"] >= 0)
predict_data["Target"] = np.where(predict_data["Target"] == True, 1, 0)
predict_data.dropna(inplace=True)
predict_data.reset_index(inplace=True,drop=True)
predict_data = predict_data[0:len(predict_data)-target_range]

In [ ]:
corr = predict_data.corr()

In [ ]:
corr

In [ ]:
corr["Target"].sort_values()

## Grouping Data by Mean

In [ ]:
new_predict_data = predict_data.copy()

In [ ]:
""""""
RSIs = ["RSI-{}".format(i) for i in range(1,max_days)]
RSIs.append("RSI")
MACDs = ["MACD-{}".format(i) for i in range(1,max_days)]
MACDs.append("MACD")
MACD_Hs = ["MACD_H-{}".format(i) for i in range(1,max_days)]
MACD_Hs.append("MACD_H")
Variations = ["Variation-{}".format(i) for i in range(1,max_days)]
Variations.append("Variation")
Revenues = ["REV-{}".format(i) for i in range(1,max_days)]
Revenues.append("Miners Revenue")
pDMs = ["+DM-{}".format(i) for i in range(1,max_days)]
pDMs.append("+DM")
mDMs = ["-DM-{}".format(i) for i in range(1,max_days)]
mDMs.append("-DM")
ADXs = ["ADX-{}".format(i) for i in range(1,max_days)]
ADXs.append("ADX14")
new_predict_data["RSI_MEAN"] = new_predict_data[RSIs].mean(axis=1)
new_predict_data["MACD_MEAN"] = new_predict_data[MACDs].mean(axis=1)
new_predict_data["MACD_H_MEAN"] = new_predict_data[MACD_Hs].mean(axis=1)
new_predict_data["Variation_MEAN"] = new_predict_data[Variations].mean(axis=1)
new_predict_data["Revenues_MEAN"] = new_predict_data[Revenues].mean(axis=1)
new_predict_data["+DM_MEAN"] = new_predict_data[pDMs].mean(axis=1)
new_predict_data["-DM_MEAN"] = new_predict_data[mDMs].mean(axis=1)
new_predict_data["ADX_MEAN"] = new_predict_data[ADXs].mean(axis=1)
new_predict_data.drop(RSIs+MACDs+MACD_Hs+Variations+Revenues+pDMs+mDMs+ADXs,axis=1,inplace=True)

In [ ]:
new_predict_data

In [ ]:
corr = new_predict_data.corr()

In [ ]:
corr

In [ ]:
corr["Target"].sort_values()

## Alternate Data

In [ ]:
alternate_predict_data = data.copy()
predict_days = 2

In [ ]:
alternate_predict_data["RSI"] = alternate_predict_data["RSI"].rolling(max_days).mean()
alternate_predict_data["MACD"] = alternate_predict_data["MACD"].rolling(max_days).mean()
alternate_predict_data["MACD_H"] = alternate_predict_data["MACD_H"].rolling(max_days).mean()
alternate_predict_data["+DM"] = alternate_predict_data["+DM"].rolling(max_days).mean()
alternate_predict_data["-DM"] = alternate_predict_data["-DM"].rolling(max_days).mean()
alternate_predict_data["ADX14"] = alternate_predict_data["ADX14"].rolling(max_days).mean()
alternate_predict_data["Transactions"] = alternate_predict_data["Transactions"].rolling(max_days).mean()
alternate_predict_data["Confirmation Time"] = alternate_predict_data["Confirmation Time"].rolling(max_days).mean()
alternate_predict_data["Miners Revenue"] = alternate_predict_data["Miners Revenue"].rolling(max_days).mean()
alternate_predict_data["FnG"] = alternate_predict_data["FnG"].rolling(max_days).mean()
alternate_predict_data["Variation"] = alternate_predict_data["Variation"].rolling(max_days).mean()
alternate_predict_data["Volume"] = alternate_predict_data["Volume"].rolling(max_days).mean()

alternate_predict_data["Target"] = (alternate_predict_data["Close"].shift(-predict_days) - alternate_predict_data["Close"] >= 0)
alternate_predict_data["Target"] = np.where(alternate_predict_data["Target"] == True, 1, 0)

In [ ]:
corr = alternate_predict_data.corr(method="spearman")

In [ ]:
corr

In [ ]:
corr["Target"].sort_values()

# 1 minute data

In [ ]:
minute_data = pd.read_csv("minute_data/BTC-USD_1M_SIGNALS.csv")

In [ ]:
minute_data["Variation+1"] = minute_data["Variation"].shift(-1)

In [ ]:
minute_data

In [ ]:
corr = minute_data.corr(method="spearman")
corr

In [ ]:
corr["Variation+1"].sort_values()

## Dollar Tick Data

In [4]:
import plotly.graph_objects as go
import pandas as pd
import mplfinance as fplt

In [14]:
data = pd.read_csv("dollar_data/dollar_data.csv", parse_dates=True)
data = data.loc[data["Unix"] >= 1609459200][0:1000]
data.index = pd.to_datetime(data['Date'])

In [15]:
data

,Unix,Date,Open,High,Low,Close
Date,,,,,,
2021-01-01 00:02:00,1609459320,2021-01-01 00:02:00,29083.47,29073.02,28938.21,28941.55
2021-01-01 00:09:00,1609459740,2021-01-01 00:09:00,28941.55,28987.85,28903.28,28925.13
2021-01-01 00:11:00,1609459860,2021-01-01 00:11:00,28925.13,28936.19,28843.74,28888.34
2021-01-01 00:12:00,1609459920,2021-01-01 00:12:00,28888.34,28931.49,28800.00,28811.01
2021-01-01 00:14:00,1609460040,2021-01-01 00:14:00,28811.01,28839.99,28774.64,28834.74
...,...,...,...,...,...,...
2021-01-02 09:32:00,1609579920,2021-01-02 09:32:00,29779.36,29807.14,29771.07,29807.10
2021-01-02 09:34:00,1609580040,2021-01-02 09:34:00,29807.10,29807.10,29757.18,29757.18
2021-01-02 09:35:00,1609580100,2021-01-02 09:35:00,29757.18,29757.18,29666.66,29704.71


In [16]:
fig = go.Figure(data=[go.Candlestick(x=data['Date'],
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close'])])

In [23]:
from matplotlib.pyplot import xticks


fig, axlist = fplt.plot(
    data,
    type='candle',
    style='charles',
    title='BTC-USD',
    ylabel='Price',
    returnfig=True
    )

ticklocations = [ data.index.get_loc(tick) for tick in data['Date'] ]
ticklabels = list(data['Date'])

print(len(ticklabels))

axlist[-2].xaxis.set_ticks(ticklocations)
axlist[-2].set_xticklabels(ticklabels)

fplt.show()


c:\Users\Alexis\AppData\Local\Programs\Python\Python38\lib\site-packages\mplfinance\_arg_validators.py:36: UserWarning:




            POSSIBLE TO SEE DETAILS (Candles, Ohlc-Bars, Etc.)
   For more information see:
   - https://github.com/matplotlib/mplfinance/wiki/Plotting-Too-Much-Data
   
   TO SILENCE THIS WARNING, set `type='line'` in `mpf.plot()`
   OR set kwarg `warn_too_much_data=N` where N is an integer 
   LARGER than the number of data points you want to plot.




1000


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\Alexis\AppData\Local\Programs\Python\Python38\lib\tkinter\__init__.py", line 1895, in __call__
    return self.func(*args)
  File "c:\Users\Alexis\AppData\Local\Programs\Python\Python38\lib\tkinter\__init__.py", line 816, in callit
    func(*args)
  File "c:\Users\Alexis\AppData\Local\Programs\Python\Python38\lib\site-packages\matplotlib\backends\_backend_tk.py", line 253, in idle_draw
    self.draw()
  File "c:\Users\Alexis\AppData\Local\Programs\Python\Python38\lib\site-packages\matplotlib\backends\backend_tkagg.py", line 9, in draw
    super(FigureCanvasTkAgg, self).draw()
  File "c:\Users\Alexis\AppData\Local\Programs\Python\Python38\lib\site-packages\matplotlib\backends\backend_agg.py", line 407, in draw
    self.figure.draw(self.renderer)
  File "c:\Users\Alexis\AppData\Local\Programs\Python\Python38\lib\site-packages\matplotlib\artist.py", line 41, in draw_wrapper
    return draw(artist, renderer, 